In [1]:
!pip install requests
!pip install beautifulsoup4
!pip install lxml

from bs4 import BeautifulSoup
from datetime import datetime
import requests
import time
import pandas as pd

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [29]:
url = 'https://services.nvd.nist.gov/rest/json/cves/2.0?cpeName=cpe:2.3:o:microsoft:windows_11_21h2:10.0.22000&cvssV3Severity=CRITICAL&resultsPerPage=50&startIndex=0&sourceIdentifier=secure@microsoft.com'
response = requests.get(url)
if response.status_code == 200:
    # Parsear la respuesta JSON
    data = response.json()
    
    # Extraer la lista de vulnerabilidades
    vulnerabilities = data.get('vulnerabilities', [])
    cvssMetrics31 = data.get('cvssMetricV31', [])
    
    # Crear una lista para almacenar los datos
    data_list = []
    
    # Iterar sobre cada vulnerabilidad y extraer la información
    for vuln in vulnerabilities:
        cve_data = vuln.get('cve', {})
        cve_id = cve_data.get('id', '')
        source_identifier = cve_data.get('sourceIdentifier', '')
        #published = vuln.get('published', '')
        #last_modified = vuln.get('lastModified', '')
        
        # Descripción
        descriptions = cve_data.get('descriptions', [])
        description = descriptions[0].get('value', '') if descriptions else ''
            
        # Métricas CVSS
        metrics = vuln.get('metrics', {})
        
        # CVSS v3.1
        cvss_v31 = metrics.get('cvssMetricV31', [])
        if cvss_v31:
            cvss_v31_data = cvss_v31[0].get('cvssData', {})
            cvss_v31_score = cvss_v31_data.get('baseScore', '')
            cvss_v31_vector = cvss_v31_data.get('vectorString', '')
        else:
            cvss_v31_score = ''
            cvss_v31_vector = ''
        
        # CVSS v2
        cvss_v2 = metrics.get('cvssMetricV2', [])
        if cvss_v2:
            cvss_v2_data = cvss_v2[0].get('cvssData', {})
            cvss_v2_score = cvss_v2_data.get('baseScore', '')
            cvss_v2_vector = cvss_v2_data.get('vectorString', '')
        else:
            cvss_v2_score = ''
            cvss_v2_vector = ''
        
        # Añadir los datos a la lista
        data_list.append({
            'CVE_ID': cve_id,
            'SourceIdentifier': source_identifier,
            #'Published': published,
            #'LastModified': last_modified,
            'Description': description,
            'CVSSv3.1_Score': cvss_v31_score,
            'CVSSv3.1_Vector': cvss_v31_vector,
            'CVSSv2_Score': cvss_v2_score
        })
    
    # Crear un DataFrame de pandas
    df = pd.DataFrame(data_list)
    
    # Seleccionar las primeras 20 filas y 8 columnas
    src = df.iloc[:20, :8]
    
    # Mostrar el resultado
    print("PuntajeScore: ",cvss_v31_score)
    print(src)
else:
    print(f"Error: {response.status_code}")
    print(response.text)


PuntajeScore:  
            CVE_ID      SourceIdentifier  \
0   CVE-2023-21708  secure@microsoft.com   
1   CVE-2023-23392  secure@microsoft.com   
2   CVE-2023-23415  secure@microsoft.com   
3   CVE-2023-21554  secure@microsoft.com   
4   CVE-2023-28250  secure@microsoft.com   
5   CVE-2023-24943  secure@microsoft.com   
6   CVE-2023-32056  secure@microsoft.com   
7   CVE-2023-32057  secure@microsoft.com   
8   CVE-2023-33154  secure@microsoft.com   
9   CVE-2023-35365  secure@microsoft.com   
10  CVE-2023-35366  secure@microsoft.com   
11  CVE-2023-35367  secure@microsoft.com   
12  CVE-2023-35385  secure@microsoft.com   
13  CVE-2023-35349  secure@microsoft.com   
14  CVE-2023-36434  secure@microsoft.com   
15  CVE-2024-30080  secure@microsoft.com   
16  CVE-2024-38063  secure@microsoft.com   
17  CVE-2024-38140  secure@microsoft.com   
18  CVE-2024-38199  secure@microsoft.com   
19  CVE-2024-21416  secure@microsoft.com   

                                          Description CVSSv